## 1. Define Problem

* For example we factorize the biprime number 187

In [98]:
from docplex.mp.model import Model

# Example to factorize 187 which is biprime
# 6 unknown bits + 6 "carry" bits = 12 qubits
    
# Create an instance of a model and variables
mdl = Model(name='qubo-fact')
x = {i: mdl.binary_var(name='x_{0}'.format(i)) for i in range(2,4 +1)}
y = {i: mdl.binary_var(name='y_{0}'.format(i)) for i in range(2,4 +1)}
#z = {i: mdl.binary_var(name='z_{0}'.format(i)) for i in range(2,6 +1)}
#z5a = mdl.binary_var(name='z_5a')

# Objective function
# mdl.minimize(x[2] + x[3] + x[4] + 2*y[2] + 2*y[3] + 2*y[4] + y[2]*x[2] + y[2]*x[3] + y[3]*x[2] + y[2]*x[4] + y[3]*x[3] + y[4]*x[2] + y[3]*x[4] + y[4]*x[3])

# Constraints
# r2:	x2 + y2 = 0 + 2z2
# r3:	x3 + y2 + y3 + z2 = 1 + 2z3
# r4:	x4 + y2x2 + y3 + y4 + z3 = 1 + 2z4 + 4z7
# r5:	y2x3 + y3x2 + y4 + z4 = 1 + 2z5
# r6:	y2x4 + y3x3 + y4x2 + z5 + z7 = 0 + 2z6
# r7:	y3x4 + y4x3 + z6 = 1

## with carries:
#dl.add_constraint(x[2] + y[2] == 0 + 2 * z[2])
#dl.add_constraint(x[3] + y[2] + y[3] + z[2] == 1 + 2 * z[3])
#dl.add_constraint(x[4] + y[2]*x[2] + y[3] + y[4] + z[3] == 1 + 2 * z[4] + 4 * z5a)
#dl.add_constraint(y[2]*x[3] + y[3]*x[2] + y[4] + z[4] == 1 + 2 * z[5])
#dl.add_constraint(y[2]*x[4] + y[3]*x[3] + y[4]*x[2] + z[5] + z5a == 0 + 2 * z[6])
#dl.add_constraint(y[3]*x[4] + y[4]*x[3] + z[6] == 1)

## without carries:
#mdl.add_constraint(x[2] + y[2] == 0)
#mdl.add_constraint(x[3] + y[2] + y[3] == 1)
#mdl.add_constraint(x[4] + y[2]*x[2] + y[3] + y[4] == 1)
#mdl.add_constraint(y[2]*x[3] + y[3]*x[2] + y[4]  == 1)
#mdl.add_constraint(y[2]*x[4] + y[3]*x[3] + y[4]*x[2] == 0)
#mdl.add_constraint(y[3]*x[4] + y[4]*x[3] == 1)

## Objective function without carries:
alpha = 4
two = 1
mdl.minimize(  alpha*(x[2] + y[2])**2
             + alpha*(x[3] + y[2] + y[3] -1)**2
             + alpha*(x[4] + y[2]*x[2] + y[3] + y[4] -1)**two
             + alpha*(y[2]*x[3] + y[3]*x[2] + y[4] -1)**two
             + alpha*(y[2]*x[4] + y[3]*x[3] + y[4]*x[2])**two
             + alpha*(y[3]*x[4] + y[4]*x[3] -1)**two
            )

print(mdl.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qubo-fact

Minimize
 obj: - 8 x_3 + 4 x_4 - 8 y_2 - 4 y_3 + 8 y_4 + [ 8 x_2^2 + 24 x_2*y_2
      + 8 x_2*y_3 + 8 x_2*y_4 + 8 x_3^2 + 24 x_3*y_2 + 24 x_3*y_3 + 8 x_3*y_4
      + 8 x_4*y_2 + 8 x_4*y_3 + 16 y_2^2 + 16 y_2*y_3 + 8 y_3^2 ]/2 -8
Subject To

Bounds
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= y_2 <= 1
 0 <= y_3 <= 1
 0 <= y_4 <= 1

Binaries
 x_2 x_3 x_4 y_2 y_3 y_4
End



## 2. Compile problem to an executable form on the chosen backend.

* Transpile to QUBO (Quadratic Unconstrained Binary Optimization) problem
* Move all conditions to the objective term via qiskit converters.


In [99]:
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import QuadraticProgramToIsing, InequalityToEquality, IntegerToBinary, LinearEqualityToPenalty

# define operators and converters
qp = QuadraticProgram()
ineq2eq = InequalityToEquality()
lineq2penalty = LinearEqualityToPenalty()

# import DOCplex model and apply converters
qp.from_docplex(mdl)
qp_eq = ineq2eq.convert(qp)
qubo = lineq2penalty.convert(qp_eq)

print(qubo.export_as_lp_string())

# Use Eigenvector based solvers
from qiskit import BasicAer
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.aqua.algorithms import QAOA, VQE, ExactEigensolver, NumPyMinimumEigensolver

# prepare Eigenvector based solvers
vqe_mes = VQE(quantum_instance=BasicAer.get_backend('statevector_simulator'))
qaoa_mes = QAOA(quantum_instance=BasicAer.get_backend('statevector_simulator'))
exact_mes = NumPyMinimumEigensolver()
vqe = MinimumEigenOptimizer(vqe_mes)   # using VQE
qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

# Example: GAS `GroverOptimizer`
from qiskit.optimization.algorithms import GroverOptimizer

# prepare GAS solver
gas_mes = BasicAer.get_backend('statevector_simulator')
grover_optimizer = GroverOptimizer(6, num_iterations=10, quantum_instance=gas_mes)


\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qubo-fact

Minimize
 obj: - 8 x_3 + 4 x_4 - 8 y_2 - 4 y_3 + 8 y_4 + [ 8 x_2^2 + 24 x_2*y_2
      + 8 x_2*y_3 + 8 x_2*y_4 + 8 x_3^2 + 24 x_3*y_2 + 24 x_3*y_3 + 8 x_3*y_4
      + 8 x_4*y_2 + 8 x_4*y_3 + 16 y_2^2 + 16 y_2*y_3 + 8 y_3^2 ]/2 -8
Subject To

Bounds
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= y_2 <= 1
 0 <= y_3 <= 1
 0 <= y_4 <= 1

Binaries
 x_2 x_3 x_4 y_2 y_3 y_4
End



## 3. Execute on the Quantum Computer backend

* simulated by CPU in this notebook for faster turnaround
* for real QC execution see below in BACKUP section

In [100]:
# solve and print
exact_result = exact.solve(qubo)
print("exact:", exact_result)

# TODO: compare to direct "mdl" solution
exact_result_mdl = mdl.solve()
print("exact:", exact_result_mdl)
# http://ibmdecisionoptimization.github.io/docplex-doc/mp/docplex.mp.model.html

qaoa_result = qaoa.solve(qubo)
print("qaoa: ", qaoa_result)
#vqe_result = vqe.solve(qubo)
#print("vqe:  ",vqe_result)

#mdl.minimize(x[2] + x[3] + x[4] + 2*y[2] + 2*y[3] + 2*y[4] + y[2]*x[2] + y[2]*x[3] + y[3]*x[2] + y[2]*x[4] + y[3]*x[3] + y[4]*x[2] + y[3]*x[4] + y[4]*x[3])
#    8 =      0      1      0      2        0        2        0           1           0           0           0           0           1           1       
# x2 x3 x4 y2 y3 y4 z2 z3 z4 z5 z6 z5a
# 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0.  <-- real solution
# 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.


exact: optimal function value: -12.0
optimal value: [0. 1. 0. 0. 0. 0.]
status: SUCCESS
exact: solution for: qubo-fact
objective: -12
x_3=1

qaoa:  optimal function value: -12.0
optimal value: [0. 1. 0. 0. 0. 0.]
status: SUCCESS


In [101]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright